In [1]:
import pandas as pd

DATA_PATH = '../data/'
filename  = 'estrepublicain_annee_1999.csv'

df = pd.read_csv(DATA_PATH + filename)
# df = df[0:1000]

# sampler 

df = df.sample(frac= 0.5)
df.reset_index(inplace = True, drop = True)
print(df.shape)

df
print("= dimensions")
print(df.shape)
print("= 5 premieres rangés")
print(df.head())
print("= Noms des colonnes")
print(df.columns)

# Changer le nom de la colonne
df.columns = ['paragraphe']
df.columns

# puis renommer colonne en 'text'

df.columns = ['text']
df.columns

# Tokenization
from nltk import word_tokenize

# La phrase
sentence = "les médiateurs confirment « l'absence de solution parfaite ». "

tokens = word_tokenize(sentence)

print(tokens)
tokens_on_space = sentence.split(' ')
print(tokens_on_space)

# ponctuation

import string

print("Liste original de signe de ponctuations")
print("\t{}".format(string.punctuation))

sentence = "les médiateurs confirment « l'absence de solution parfaite ». "
punctuation_chars = string.punctuation + "«»"

#  Construction d'un dict { '~':' ', '$': ' ', ... }
dict_ponctuation = {}
for k in punctuation_chars:
    dict_ponctuation[k] = ' '

print(dict_ponctuation)

# # L'operateur de translation
translator = str.maketrans(dict_ponctuation)

new_sentence = sentence.translate(translator)
print(sentence)
print(new_sentence)

# stopwords
tokens = word_tokenize(new_sentence)
print(tokens)

from nltk.corpus import stopwords


# print("=== stopwords - français:")
# print(sorted(stopwords.words('french')))

list_stopwords = stopwords.words('french') + ['les', 'de']

# # equivalent:
# stopwords = stopwords.words('french')
# stopwords.append('les')
# stopwords.append('de')


# 
tokens_sans_stopwords = [w for w in tokens if (w not in list_stopwords) ]

## Equivalent 
tokens_sans_stopwords = []
for w in tokens:
    if w not in list_stopwords:
        tokens_sans_stopwords.append(w)



print(tokens_sans_stopwords)

# Enlevons la ponctuation

df['text_no_punctuation'] = df.text.apply(lambda 
    r : ( r.translate(translator) ) 
)
df.head()

# Tokenizer

df['tokens_all']  = df.text_no_punctuation.apply(
    lambda r : word_tokenize(r.lower())
)

df.head()

# remove stopwords

def remove_stopword(tokens):
     return [w for w in tokens if (w not in list_stopwords) ]

# Verifier que ca marche
remove_stopword(tokens)

# appliquer a la dataframe

df['tokens'] = df.tokens_all.apply(
    lambda tks : remove_stopword(tks) 
)

df.head()

# enlever les colonnes intermediaires

print(df.columns)

df = df[['text', 'tokens']]

print(df.columns)

# nombre de tokens par rangée
pd.options.mode.chained_assignment = None
df['token_count'] = df.tokens.apply( lambda r : len(r) )

print(df.head())

# repartition du nombre de tokens
df.token_count.describe()

# quel document a 955 tokens?

condition = (df.token_count == 955)
print(df[condition].text.values)

# quel index
print(df[condition].index)

# 
condition = (df.token_count < 800)
print(df[condition].shape)

# enlever le paragraphe le plus long

condition_filtrage = df.token_count < 955
print(df[condition_filtrage].shape)

df = df[condition_filtrage]
print(df[condition_filtrage].shape)

print(df.token_count.describe())

# Gensim - Vocabulaire

from gensim import corpora, models
dictionary  = corpora.Dictionary(df.tokens)
print(dictionary)

# corpus_gensim

df['corpus_gensim'] = df.tokens.apply(lambda d : dictionary.doc2bow(d))

print(df[['text','corpus_gensim']] .head().values)

corpus_gensim = [c for c in df.corpus_gensim ]

num_topics= 40

# Le model LDA
lda = models.LdaModel(corpus_gensim,
    id2word      = dictionary,
    num_topics   = num_topics,
    alpha        = 'asymmetric',
    eta          = 'auto',
    passes       = 2,
    iterations   = 20
)

for t in lda.show_topics(num_topics=num_topics, formatted=True, log = False):
    print("\n=== topic #{}".format(t[0]))
    print(t[1].replace('*', ': ').replace(' +',', ').replace('"',''))


(15120, 1)
= dimensions
(15120, 1)
= 5 premieres rangés
                                                text
0  Le député maire, Claude Jacquot, et le maire J...
1  Pour la première fois, Le Val d'Ajol recevait ...
2  Depuis la fin des années 60 en effet, la ville...
3  « La leçon à retenir est qu'il faut être plus ...
4  BESANÇON._ Le sénateur UDF de Haute-Saône, Ber...
= Noms des colonnes
Index(['text'], dtype='object')
['les', 'médiateurs', 'confirment', '«', "l'absence", 'de', 'solution', 'parfaite', '»', '.']
['les', 'médiateurs', 'confirment', '«', "l'absence", 'de', 'solution', 'parfaite', '».', '']
Liste original de signe de ponctuations
	!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{'!': ' ', '"': ' ', '#': ' ', '$': ' ', '%': ' ', '&': ' ', "'": ' ', '(': ' ', ')': ' ', '*': ' ', '+': ' ', ',': ' ', '-': ' ', '.': ' ', '/': ' ', ':': ' ', ';': ' ', '<': ' ', '=': ' ', '>': ' ', '?': ' ', '@': ' ', '[': ' ', '\\': ' ', ']': ' ', '^': ' ', '_': ' ', '`': ' ', '{': ' ', '|': ' ', '}': ' ', 

/home/alexis/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:165: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Dictionary(81946 unique tokens: ['député', 'maire', 'claude', 'jacquot', 'jacky']...)
[[ "Le député maire, Claude Jacquot, et le maire Jacky Chozerot se sont accordés à reconnaître l'esprit inventif des enfants. C'est avec beaucoup d'émotion dans la voix que Chantal Gérard a dédié cette exposition à Robert George qui fut, a-t-elle dit, un excellent pédagogue, un professeur d'arts plastiques remarquable, un grand artiste, un grand homme. Avant de remercier son fils Jean-Claude George, « grâce auquel je me suis lancée dans ce projet » a-t-elle poursuivi, « merci aussi à Mme Robert, conseillère pédagogique en arts plastiques, à M. Piesch pour leurs excellents conseils et merci surtout aux enfants qui se sont investis toute l'année et qui sont aujourd'hui les maîtres d'œuvres de cette exposition »."
  list([(0, 1), (1, 2), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 3), (17, 1), (18, 2), (19, 2), (20, 2), (21, 2